In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [6]:
# Read the data into a dataframe
raw_data = pd.read_csv('UnemploymentData.csv', index_col = 0).drop(0, axis=0)

In [3]:
#Set the index as the State and drop unwanted rows 
ue_data = raw_data.set_index('State')
ue_data = ue_data.drop('PR', axis=0)
ue_data = ue_data.reset_index()

In [4]:
# Create a list of the desired columns 
cols = []
cols.append('State')
cols.append('Area_name')
cols.append('Median_Household_Income_2016')
# Extract all the columns containing unemployment rates
for i in ue_data.columns:
    if 'Unemployment_rate' in i:
        cols.append(i)

In [5]:
ue_data.shape

(3195, 51)

In [6]:
# Drop rows containing null values
ue_data = ue_data[cols].dropna(how='any')
ue_data.head()

,State,Area_name,Median_Household_Income_2016,Unemployment_rate_2007,Unemployment_rate_2008,Unemployment_rate_2009,Unemployment_rate_2010,Unemployment_rate_2011,Unemployment_rate_2012,Unemployment_rate_2013,Unemployment_rate_2014,Unemployment_rate_2015,Unemployment_rate_2016,Unemployment_rate_2017
0,AL,Alabama,"$46,309",4.0,5.7,11.0,10.5,9.6,8.0,7.2,6.8,6.0,5.9,4.4
1,AL,"Autauga County, AL","$54,487",3.3,5.1,9.7,8.9,8.4,6.9,6.2,5.8,5.0,5.1,3.9
2,AL,"Baldwin County, AL","$56,460",3.1,4.6,9.8,10.0,9.0,7.5,6.6,6.1,6.0,5.4,4.0
3,AL,"Barbour County, AL","$32,884",6.3,8.8,14.3,12.3,11.5,11.5,10.2,10.5,9.0,8.4,5.9
4,AL,"Bibb County, AL","$43,079",4.1,5.8,13.3,11.4,10.5,8.5,7.9,7.2,7.0,6.5,4.4


In [7]:
# Rename the columns
ue_data.columns = ['State', 'Area', 'MedianIncome2016', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
                  '2015', '2016', '2017']

In [8]:
# Convert the median income to int values
m = []
for val in ue_data.MedianIncome2016:
    val = str(val)
    newval = re.sub(r'\D', "", val) # remove non-digits
    m.append(int(newval))

income = pd.Series(m)
ue_data.MedianIncome2016 = income

In [9]:
stacked_data = ue_data.set_index(['State', 'Area', 'MedianIncome2016']).stack()

In [10]:
# Stack the data so that there is only one column for the unemployment rate and one column for the year
stacked_data = stacked_data.reset_index()
stacked_data.columns = ['State' ,'Area', 'MedianIncome2016', 'Year', 'UnemploymentRate']

In [11]:
# Set the median income value to null for all years other than 2016, since we have only data for 2016
new_vals = []
for i in stacked_data.index:
    if str(stacked_data.Year[i]) == '2016':
        new_vals.append(stacked_data.MedianIncome2016[i])
    else:
        new_vals.append(np.nan)
        
stacked_data.MedianIncome2016 = new_vals

In [12]:
# Regions of the U.S.
L1 = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA']
L2 = ['DE','FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'DC', 'WV', 'AL', 'KY', 'TN', 'LA', 'OK', 'TX', 'MS', 'AR']
L3 = ['IL', 'IN', 'MI', 'OH', 'WI', 'IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD']
L4 = ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT', 'WY', 'AK', 'CA', 'HI', 'OR', 'WA']

In [13]:
# Create a column for the region
region = []
for i in stacked_data.index:
    if str(stacked_data.State[i]) in L1:
        region.append('Northeast')
    elif str(stacked_data.State[i]) in L2:
        region.append('South')
    elif str(stacked_data.State[i]) in L3:
        region.append('Midwest')
    elif str(stacked_data.State[i]) in L4:
        region.append('West')
    else:
        region.append(np.nan)
        
region = pd.Series(region)
stacked_data['Region'] = region.values

In [14]:
area = []
for i in stacked_data.index:
    a = stacked_data.Area[i]
    s = a.split(',')
    area.append(s[0])

In [15]:
stacked_data.Area = area

In [16]:
stacked_data.to_csv('CleanDataUnemployment.csv')